In [28]:

#                          Movies Recommendation System     ->   Content-Based Filtering

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

movies = pd.read_csv('20__5000_movies_practice.csv')
credits = pd.read_csv('20__tmdb_5000_credits.csv')

movies = movies.merge(credits,on='title')
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

movies.dropna(inplace=True)  # Drop null values

def fetch_director(text):
    L = []
    for i in eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 
def convert3(text):
    L = []
    counter = 0
    for i in eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 
def convert(text):
    L = []
    for i in eval(text):
        L.append(i['name']) 
    return L 

movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert)
movies['cast'] = movies['cast'].apply(lambda x:x[0:3])
movies['crew'] = movies['crew'].apply(fetch_director)


# movies.head()

def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)


# movies.head()

movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])
#new.head()
new['tags'] = new['tags'].apply(lambda x: " ".join(x).lower())
new.head()

# Summary => combined all rows  from all columns under tags column
#            so we just have tags and title column in new df 

#     TESTING
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
    
vector = cv.fit_transform(new['tags']).toarray()  # Tags column converted into array 

# vector                                    #  tags column converted into numpy array 

from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)    # we check similarity of each vectory or array with each other 

similarity[0]  # show similarity for movie Avatar with respect to each movie in vector

# list(enumerate(similarity[index]))  # since similarity are all the movies ,we give index to each movie manually
# now we use sorted -> sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1]) 
# so that we get similar movies based on max similarity score 
# since all scores of movie[0] will be fetched out from list of all other movies( ie tags )

# EXAMPLE 
# new[new['title'] == 'The Lego Movie']
# new[new['movie_id'] == 12]
new.iloc   # index position of each movie in the array
def recommend(movie):
    index = new[new['title'] == movie].index[0]    # Converting Input from user into number 
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1]) # get the results by passing into similarity
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)    

# recommend('Gandhi')  
# Got the Recommended movies based on similarity score from similarity
    
# import joblib
# joblib.dump(new,'similar_movies.pkl')    


744